get upcoming matchups from ufcstats

In [ ]:
import scrapy
import json
from bs4 import BeautifulSoup
import re
import logging



class BoutSpider(scrapy.Spider):
    name = "upcoming_bouts_spider"
    allowed_domains = ["ufcstats.com"]
    # Set up logging
    custom_settings = {
        'LOG_FILE': 'upcoming_spider_log.txt',  # Specify the log file path
        'LOG_LEVEL': 'DEBUG',  # Set the log level to DEBUG
    }


    def start_requests(self):
        url = "http://ufcstats.com/statistics/events/upcoming"
        yield scrapy.Request(url, callback=self.get_latest_event)

    def get_upcoming_events(self, response):
        upcoming_event_urls = response.css('tr.b-statistics__table-row a::attr(href)').getall()
        for url in upcoming_event_urls:
            yield scrapy.Request(url, callback=self.get_bout_links)

    def get_bouts(self, response):
        bouts = response.css('tr.b-fight-details__table-row').getall()
        for bout in range(1, len(bouts) + 1):
            event_title = response.css('span.b-content__title-highlight::text').get().strip()
            date = response.xpath('string(//li[@class="b-list__box-list-item"]/i[@class="b-list__box-item-title"][contains(text(), "Date:")]/following-sibling::text()[1])').get().strip()
            location = response.xpath('string(//li[@class="b-list__box-list-item"]/i[@class="b-list__box-item-title"][contains(text(), "Location:")]/following-sibling::text()[1])').get().strip()
            fighter_a_name = response.css(f'tr.b-fight-details__table-row:nth-child({bout}) td.b-fight-details__table-col.l-page_align_left:nth-child(2) p.b-fight-details__table-text:nth-child(1) a.b-link::text').get().strip()
            fighter_b_name = response.css(f'tr.b-fight-details__table-row:nth-child({bout}) td.b-fight-details__table-col.l-page_align_left:nth-child(2) p.b-fight-details__table-text:nth-child(2) a.b-link::text').get().strip()
            # response.css('tr.b-fight-details__table-row:nth-child(1) td.b-fight-details__table-col.l-page_align_left:nth-child(2) p.b-fight-details__table-text:nth-child(2) a.b-link::text').get().strip()
            weight_class = response.css(f'tr.b-fight-details__table-row:nth-child({bout}) td.b-fight-details__table-col:nth-child(7) p.b-fight-details__table-text::text').get().strip()
            #response.css('tr.b-fight-details__table-row:nth-child(1) td.b-fight-details__table-col:nth-child(7) p.b-fight-details__table-text::text').get().strip()
            
            #if event_title does not contain 'Fight Night' and location does not contain 'Vegas':
            if 'Fight Night' not in event_title and 'Vegas' not in location:
                cage_size = 'big'
            else:
                cage_size = 'small'



            yield {
                'event_title': event_title,
                'date': date,
                'location': location,
                'fighter_a_name': fighter_a_name,
                'fighter_b_name': fighter_b_name,
                'weight_class': weight_class,
                'cage_size': cage_size,
            }
